# Personal Search

## Context search

This code aims to perform a search in texts loaded in the vector-store. The result seeks to be more of a quote than an elaborate answer. It will be used later to answer questions.

## Busca de contexto

Esse código tem como objetivo realizar uma busca em textos carregados no vector-store. O resultado busca ser mais uma citação do que uma resposta elaborada. Será usada mais a frente para responder perguntas.

## Import packages

In [9]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_community.llms import Ollama
from IPython.display import display, Markdown
import logging
import os
from langchain import hub

## Configuration

In [10]:
# Model from Ollama
model = "llama3:8b"
# Embeddings Model
embeddings_model = "nomic-embed-text:v1.5"
# Vectore Store path
path_books = "./vector-store/books"

## Retriever Instance

In [11]:
# Load the embeddings model
embeddings = OllamaEmbeddings(model=embeddings_model)
# Load the Vector Store
db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

# Load vectore store as retriever (search engine)
retriever = db.as_retriever()
# distance_metric: cosine
retriever.search_kwargs["distance_metric"] = "cos"
# fetch_k: search method to set how many documents you want to fetch before filtering
retriever.search_kwargs["fetch_k"] = 100
# maximal_marginal_relevance: The MaxMarginalRelevanceExampleSelector selects 
# examples based on a combination of which examples are most similar to the inputs,
# while also optimizing for diversity.
retriever.search_kwargs["maximal_marginal_relevance"] = False
# k: Number of documents to return
retriever.search_kwargs["k"] = 3

## Prompt

In [13]:
# Validated Prompt 
prompt = hub.pull(owner_repo_commit="personal-search/context-search")

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'personal-search', 'lc_hub_repo': 'context-search', 'lc_hub_commit_hash': '566cc2d5577b4fdb837c190d3eade0b0bf3a84422180b303835700c9a0d6228c'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nContexto:\n{context} \n\nCom base apenas no contexto, traga trechos que tenham relação com o assunto "{question}". \nSe não houver relação, diga que não há.\nResponda sempre em português.\n'))])

## Question

In [14]:
question = "O que falta aos pregadores de hoje?"

## Search

In [15]:
llm = Ollama(model=model, temperature=0)

final_rag_chain = (
    {"context": retriever, 
     "question": itemgetter("question")} 
    | prompt
    | llm.bind(stop=['<|eot_id|>']) # stop when the model outputs the end of text token (llama3)
    | StrOutputParser()
)

final_answer = final_rag_chain.invoke({"question":question})

## Anwser

In [16]:
display(Markdown(f"**Busca:** {question}\n\n"))
display(Markdown(final_answer))

**Busca:** O que falta aos pregadores de hoje?



A partir do contexto fornecido, encontrei os seguintes trechos relacionados ao assunto "O que falta aos pregadores de hoje?" :

* Do documento com o título "Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf" (página 45):
"Pentecostes significa dor, mas o que ma is expe rimentamos é prazer; significa peso; mas nós amamos a comodidade. Pentecostes significa prisão, e, no entanto, a maioria dos crentes faria qualquer coisa, menos ir para a prisão por amor a Cristo."

* Do documento com o título "Conselhos para obreiros - Charles H Spurgeon.pdf" (página 11):
"Nós precisamos nos enver gonhar de não sentir vergonha; precisamos chorar por n ão termos lágrimas; precisamos nos humilhar por haver perdido a humildade de servo de Deus; gemer por não sentirmos peso pelos perdi dos; irar-nos contra n ós mesmos por não termos ódio do monopólio que o diabo exerce nestes dias do fim, e nos punir pelo fato de o mundo estar-se dando tão bem conosco, que nem precisa perseguir-nos."

Esses trechos sugerem que os pregadores de hoje em dia podem estar falta de características como humildade, dor, lágrimas e amor a Cristo, o que pode estar afetando sua eficácia na pregação.

## Debug

In [6]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Trecho:** pregadores, precisamos nos enver gonhar de não sentir vergonha; 
precisamos chorar por n ão termos lágrimas; precisamos nos humilhar 
por haver perdido a humildade de servo de Deus; gemer por não 
sentirmos peso pelos perdi dos; irar -nos contra n ós mesmos por não 
termos ódio do monopólio que o diabo exerce nestes dias do fim, e 
nos punir pelo fato de o mundo estar -se dando tão bem conosco, que 
nem precisa perseguir -nos. 
Pentecostes significa dor, mas o que ma is expe rimentamos  é 
prazer; significa peso; mas nós amamos a comodidade. Pentecostes 
significa prisão, e, no entanto, a maioria dos crentes faria qualquer 
coisa, menos ir para a prisão por amor a Cristo. Se revivêss emos a 
experiência do pentecostes, talvez muitos de nós fossem parar na

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 77}

**Trecho:** na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem 
tem coração quebrantado é capaz de levar outros ao 
quebrantamento. Ir mãos, nós temos equipamentos, mas não temos 
poder; temos ação, mas não unção; barulho,  mas não avivamento. 
Somos dogmát icos, mas  não dinâmicos!  
Todas as eras têm iniciado com fogo, e todas as vidas, sejam de

**Fonte:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 11}

**Trecho:** “Pensei que ele não havia se preocupado muito conosco, porque sabia que, por onde quer que ele
passe, argumenta com seus anfitriões, e não havia feito assim conosco. Cheguei a pensar que
havíamos irritado o sr. Whitefield, mas agora percebo como ele foi amável conosco, falando-nos
dessa maneira.”
Em seguida chamou suas filhas: “Subam, meninas. Vejam o que o homem de Deus escreveu na
vidraça: ‘Falta-lhes uma coisa!’. Chamem seu pai”. O coronel subiu ao aposento de hóspedes e
também leu a frase: “Falta-lhes uma coisa!”. Assim, ao redor da cama onde o homem de Deus
havia dormido, ajoelharam-se e pediram a Deus que lhes desse a coisa que faltava. E, ali mesmo,
antes  que deixassem o quarto, encontraram  o que lhes  faltava;  e toda aquela  família  pôde
regozijar-se em Jesus.
Não faz muito tempo, encontrei-me com um amigo em cuja igreja há uma irmã que preserva
aquele pedaço de vidro como herança de família. Se você não conseguir aconselhar ou admoestar

In [7]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))

**Fonte:** pregadores, precisamos nos enver gonhar de não sentir vergonha; 
precisamos chorar por n ão termos lágrimas; precisamos nos humilhar 
por haver perdido a humildade de servo de Deus; gemer por não 
sentirmos peso pelos perdi dos; irar -nos contra n ós mesmos por não 
termos ódio do monopólio que o diabo exerce nestes dias do fim, e 
nos punir pelo fato de o mundo estar -se dando tão bem conosco, que 
nem precisa perseguir -nos. 
Pentecostes significa dor, mas o que ma is expe rimentamos  é 
prazer; significa peso; mas nós amamos a comodidade. Pentecostes 
significa prisão, e, no entanto, a maioria dos crentes faria qualquer 
coisa, menos ir para a prisão por amor a Cristo. Se revivêss emos a 
experiência do pentecostes, talvez muitos de nós fossem parar na

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Score:** 398.0030822753906

**Fonte:** na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem 
tem coração quebrantado é capaz de levar outros ao 
quebrantamento. Ir mãos, nós temos equipamentos, mas não temos 
poder; temos ação, mas não unção; barulho,  mas não avivamento. 
Somos dogmát icos, mas  não dinâmicos!  
Todas as eras têm iniciado com fogo, e todas as vidas, sejam de

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 77}

...

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 11}

**Score:** 410.0894775390625

**Fonte:** ela mesma. 
Se Jesus tivesse dito a ela antes de fazê-la: "Há apenas duas vagas disponíveis para você. Você pode ser uma negra ou então uma branca pobre. Qual dessas?", ela teria se retraído e se contorcido antes de finalmente responder: "Tudo bem, faça-me negra então, mas não uma dessas maloqueiras". E el e a teria feito uma mulher negra limpa e 
asseada, ela mesma numa versão negra. 
 Bem, um dia a sra. Turpin vai ao consultório médico e vê-se cercada pelo tipo de pessoas que ela despreza. De repente uma jovem com acne atra vessa a passos largos a sala de espera, golpeia 
a sra. Turpin com um livro e tenta estrangulá-l a. Quando a ordem é finalmente restaurada, a 
sra. Turpin, como se estivesse ag uardando uma revelação, pergunta: 
— O que você tem a me dizer? 
— Volte para o inferno de onde você ve io, sua porca velha! — grita a moça. 
A sra. Turpin desaba. O seu mundo fez-se em pedaços. O Deus que ela havia moldado à

**Trecho:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 18}

**Score:** 416.1697692871094